In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *

In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cgreg\AppData\Local\Temp\ipykernel_93724\4219252773.py:1: SyntaxWarning: invalid escape sequence '\d'
  x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")


In [4]:
#Turn y into 0s and 1s
y_tr_ = (y_train + 1) / 2
y_tr = y_tr_[:240000]
y_te = y_tr_[240000:300000]
print(y_tr.shape, y_te.shape, y_tr, y_te)
#print(x_test[0])

(240000,) (60000,) [0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]


In [79]:
my_features = [26,27,28,30,33,34,35,37,39,40,41,42,43,45,46,47,48,62,63,65]
j = 65 
print(x_train[:,j][:5])
print(x_train[0][:5])

count = 0 
for i in x_train[j]:
    if i == 72:
        count += 1

print(count/len(x_train[j]))

[1. 2. 2. 2. 2.]
[5.3000000e+01 1.1000000e+01 1.1162015e+07 1.1000000e+01 1.6000000e+01]
0.003115264797507788


In [80]:

def remove_categorical_features(X, threshold=10):
    """
    Removes categorical (low-cardinality) features from a numeric dataset.
    A feature is dropped if it has fewer than `threshold` unique values.
    """
    X_np = np.asarray(X)
    n_samples, n_features = X_np.shape
    keep_mask = np.array([
        np.unique(X_np[:, j]).size >= threshold for j in range(n_features)
    ])
    X_num = X_np[:, keep_mask]
    return X_num, keep_mask


def anova_f_test(X, y):
    """
    Compute ANOVA F-statistic for each feature using NumPy only.
    
    Parameters
    ----------
    X : np.ndarray, shape (n_samples, n_features)
        Feature matrix.
    y : np.ndarray, shape (n_samples,)
        Class labels.
    
    Returns
    -------
    F_values : np.ndarray, shape (n_features,)
        F-statistics for each feature.
    """
    n_samples, n_features = X.shape
    classes = np.unique(y)
    k = len(classes)
    
    overall_means = np.nanmean(X, axis=0)  # handle NaNs safely
    
    # Initialize sums of squares
    ssb = np.zeros(n_features)
    ssw = np.zeros(n_features)
    
    for c in classes:
        X_c = X[y == c]
        n_c = X_c.shape[0]
        mean_c = np.nanmean(X_c, axis=0)
        
        ssb += n_c * (mean_c - overall_means) ** 2
        ssw += np.nansum((X_c - mean_c) ** 2, axis=0)
    
    # Avoid division by zero
    ssw = np.where(ssw == 0, np.nan, ssw)
    
    F = (ssb / (k - 1)) / (ssw / (n_samples - k))
    
    return F


In [81]:
x_cat = remove_categorical_features(x_train, 30)
print(x_cat[0].shape)

(328135, 76)


In [83]:
# --- CLEANING ---
"""
# 0. Remove categorical (low-cardinality) features
x_num, cat_mask = remove_categorical_features(x_train, threshold=10)
print(x_num.shape)
"""
# 1. Remove features with too many NaNs
x_clean, keep_mask = remove_nan_features(x_train)

x_clean_red = x_clean[:,my_features]

x_clean_tr = x_clean_red[:240000]
x_clean_te = x_clean_red[240000:300000]


# 2. Impute remaining missing values
# Compute feature means from the training data (ignore NaNs)
train_means = np.nanmean(x_clean_tr, axis=0)

train_stds = np.nanstd(x_clean_tr, axis = 0)
# Replace NaNs in training data with training means
inds_tr = np.where(np.isnan(x_clean_tr))
x_clean_tr[inds_tr] = np.take(train_means, inds_tr[1])

# Replace NaNs in test data with training means
inds_te = np.where(np.isnan(x_clean_te))
x_clean_te[inds_te] = np.take(train_means, inds_te[1])
"""
# Compute F-scores using only the training data
F_scores = anova_f_test(x_clean_tr, y_tr)


# Select top 30 features
top_k = 
top_features_idx = np.argsort(F_scores)[-top_k:][::-1]

# Apply the same feature selection to all sets
x_anova_tr = x_clean_tr[:, top_features_idx]
x_anova_te = x_clean_te[:, top_features_idx]
"""
# --- STANDARDIZATION ---
x_tr_st = (x_clean_tr - train_means) / train_stds
x_te_st = (x_clean_te - train_means) / train_stds


print(x_tr_st.shape, y_tr.shape)
print(np.sum(y_tr))

(240000, 20) (240000,)
21131.0


In [84]:
def nearest_neighbour(x,tx,k):

    k = min(k,len(tx))
    dist = 10000*np.ones(k)
    max_dist = 10000
    NNeigh = [56]*k

    for index , x_temp in enumerate(tx):
        temp_dist = np.transpose(x_temp-x)@(x_temp-x)
        if temp_dist < max_dist:
            index_oldmax = np.where(dist == max_dist)[0][0]
            dist[index_oldmax] = temp_dist
            NNeigh[index_oldmax] = int(index)
            max_dist = np.max(dist)

    return NNeigh
            
    


In [85]:
x_nn = np.array([0,1])
tx_nn = np.array([[0,2],[1,11],[1,2],[3,0]])

res = nearest_neighbour(x_nn,tx_nn,3)

print(res)

[0, 3, 2]


In [86]:
x_tr_ba, y_tr_ba = balance_data(x_tr_st, y_tr)
print(x_tr_ba.shape, y_tr_ba.shape)


(42262, 20) (42262,)


In [87]:
near_neigh = nearest_neighbour(x_te_st[0],x_tr_st,2)
print(near_neigh,x_tr_st.shape)

[23833, 164728] (240000, 20)


In [88]:
print(1-int(0.6 < 0.5))

1


In [89]:
def classify_nneigh(xte,tx,ytr,k):
    classes = 2*np.ones(len(xte))
    for i,x in enumerate(xte):
        nneighs = nearest_neighbour(x,tx,k)
        val = np.mean(ytr[nneighs])
        classes[i] = 1-int(val < 0.5)
    
    return classes

In [90]:
x_nn = np.array([[0,1],[1,12]])
tx_nn = np.array([[0,2],[1,11],[1,2],[3,0]])
ytr = np.array([0,1,0,1])

res = classify_nneigh(x_nn,tx_nn,ytr,3)

print(res)

[0. 0.]


In [101]:
y_pred_nn = classify_nneigh(x_te_st[:1000],x_tr_st,y_tr,3)

In [105]:
l = 1000
#print(y_pred_nn, y_te[:l],len(x_te_st))

n_errors = np.sum(np.abs(y_te[:l] - y_pred_nn))
accuracy = np.mean(y_te[:l] == y_pred_nn)

print("Misclassified samples:", n_errors)
print(f"Accuracy: {accuracy:.3f}")
print(np.sum(y_pred_nn))
print(np.sum(y_te[:l]))

Misclassified samples: 107.0
Accuracy: 0.893
41.0
80.0


In [ ]:
#create_csv_submission(test_ids, y_pred, 'submission_1510')

In [106]:
def f1_score(y_true, y_pred):
    """Compute F1 score between true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [107]:
print(f1_score(y_te[:l], y_pred_nn[:l]))

0.11570247933884296
